In [ ]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']
period = params['periods']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
summary_dir = params['summary_dir']

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
best_path_skim_dir = params['best_path_skim_dir']

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [ ]:
df_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster.parquet'))

In [ ]:
df_trn = df_trips.loc[df_trips['trip_mode'].isin([6,7,8])]

In [ ]:
create_rail_crowding_od_pairs(preprocess_dir, transit_skims_dir, period, acc_egg_modes)

In [ ]:
df_temp = []

for period in time_periods:
    print(f'processing - {period}')
    
    df_od_pr = omx.open_file(_join(preprocess_dir, "rail_od_v9_trim_" + period.upper() + ".omx"))
    df_od_cwd = omx.open_file(_join(preprocess_dir, "rail_crowding_od_v9_trim_" + period.upper() + ".omx"))
    df_trn_pd = df_trn[df_trn['Period'] == period]
    
    #walk transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'WALK_TRANSIT']
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_wlk = pd.merge(df_trn_acc, df_rail_od, 
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    df_trn_wlk = pd.merge(df_trn_wlk, df_rail_cwd,
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    #print(df_trn_wlk.columns)
    
    # PNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'PNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_PNR')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_PNR', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    #del df_trn_pnr_inb['orig']
    #del df_trn_pnr_inb['dest']
    
    df_trn_pnr_inb = pd.merge(df_trn_pnr_inb, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    
    #print(df_trn_pnr_inb.columns)
    
    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'PNR_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'PNR_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    
    #del 
    df_trn_pnr_outbnd = pd.merge(df_trn_pnr_outbnd, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    
    #print(df_trn_pnr_outbnd.columns)

    df_trn_pnr = pd.concat([df_trn_pnr_inb, df_trn_pnr_outbnd], ignore_index=True)
    
    # KNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'KNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_KNR', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_KNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_knr_inb = pd.merge(df_trn_knr_inb, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')


    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'KNR_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'KNR_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_knr_outbnd = pd.merge(df_trn_knr_outbnd, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_knr = pd.concat([df_trn_knr_inb, df_trn_knr_outbnd], ignore_index=True)
    
    df_trn_rail = pd.concat([df_trn_wlk, df_trn_pnr, df_trn_knr], ignore_index=True)
    df_temp.append(df_trn_rail)

df_trn_rail = pd.concat(df_temp)
print(df_trn_rail.columns)

In [ ]:
df_trn_rail['crowd_trips'] = df_trn_rail['trips']*df_trn_rail['crowd']/100

In [ ]:
df_trn_rail['pp_trips'] = df_trn_rail['trips']*df_trn_rail['pp_share']/100
df_trn_rail['crowd_pp_trips'] = df_trn_rail['pp_trips']*df_trn_rail['crowd']/100

In [ ]:
df1 = df_trn_rail.groupby(['Period'])['crowd_trips'].sum().reset_index()
df2 = df_trn_rail.groupby(['Period'])['trips'].sum().reset_index()

df = pd.merge(df1, df2, on = 'Period', how='left')
df['Value'] = df['crowd_trips'] / df['trips']

region_value = df[['Period', 'Value']]

In [ ]:
# summarise for prioirty population
#regional value

#region_value = df_trn_rail.groupby(['Period'])['trips'].mean().reset_index()
#region_value = region_value.rename(columns={'trips': 'Value'})
region_value['Concept_ID'] = concept_id
region_value['Metric_ID'] = 'A1.10'
region_value['Metric_name'] = 'Crowding (Region)'
region_value['Submetric'] = 'A1.10.1'
region_value['Description'] = 'Average regional crowding level'
region_value['Population'] = 'Whole Population'
region_value['Geography'] = 'Regional'
region_value['Origin_zone'] = ''
region_value['Dest_zone'] = ''
region_value['Purpose'] = ''
region_value['Mode'] = ''
region_value['Income'] = ''
region_value['Zone_ID'] = ''
region_value['Units'] = 'minutes'
region_value['Total_Increment'] = ''
region_value = region_value[perf_measure_columns]


region_value_all = pd.DataFrame(columns = perf_measure_columns)

region_value_all = region_value_all.append({'Concept_ID' : concept_id,
                      'Metric_ID' : 'A1.10',
                      'Metric_name' : 'Crowding (Region)',
                      'Submetric' : 'A1.10.1',
                      'Description' : 'Average regional crowding level',
                      'Population' : 'Whole Population',
                      'Period' : 'All',
                      'Geography' : 'Regional',
                      'Zone_ID' : '',
                      'Origin_zone' : '',
                      'Dest_zone' : '',
                      'Purpose' : '',
                      'Mode' : '',
                      'Income' : '',
                      'Value' : df_trn_rail['crowd_trips'].sum()/df_trn_rail['trips'].sum(),
                      'Units' : 'minutes',
                      'Total_Increment' : ''}, ignore_index=True)


region_value_all = region_value_all[perf_measure_columns]

In [ ]:
final_df1 = pd.concat([region_value, region_value_all])
final_df1

In [ ]:
final_df1.to_csv(_join(summary_dir, "A1.10.1_regional_crowding_" + concept_id + '_region' + filename_extension + ".csv"), index=False)

In [ ]:
df1 = df_trn_rail.groupby(['Period'])['crowd_pp_trips'].sum().reset_index()
df2 = df_trn_rail.groupby(['Period'])['pp_trips'].sum().reset_index()

df = pd.merge(df1, df2, on = 'Period', how='left')
df['Value'] = df['crowd_pp_trips'] / df['pp_trips']

region_value = df[['Period', 'Value']]

In [ ]:
# summarise for prioirty population
#regional value


#region_value = df_trn_rail.groupby(['Period'])['pp_trips'].mean().reset_index()
#region_value = region_value.rename(columns={'pp_trips': 'Value'})
region_value['Concept_ID'] = concept_id
region_value['Metric_ID'] = 'A1.10'
region_value['Metric_name'] = 'Crowding (Region)'
region_value['Submetric'] = 'A1.10.2'
region_value['Description'] = 'Average regional crowding level'
region_value['Population'] = 'Prioirty Population'
region_value['Geography'] = 'Regional'
region_value['Origin_zone'] = ''
region_value['Dest_zone'] = ''
region_value['Purpose'] = ''
region_value['Mode'] = ''
region_value['Income'] = ''
region_value['Zone_ID'] = ''
region_value['Units'] = 'minutes'
region_value['Total_Increment'] = ''
region_value = region_value[perf_measure_columns]


region_value_all = pd.DataFrame(columns = perf_measure_columns)

region_value_all = region_value_all.append({'Concept_ID' : concept_id,
                      'Metric_ID' : 'A1.10',
                      'Metric_name' : 'Crowding (Region)',
                      'Submetric' : 'A1.10.2',
                      'Description' : 'Average regional crowding level',
                      'Population' : 'Prioirty Population',
                      'Period' : 'All',
                      'Geography' : 'Regional',
                      'Zone_ID' : '',
                      'Origin_zone' : '',
                      'Dest_zone' : '',
                      'Purpose' : '',
                      'Mode' : '',
                      'Income' : '',
                      'Value' : df_trn_rail['crowd_pp_trips'].sum()/df_trn_rail['pp_trips'].sum(),
                      'Units' : 'minutes',
                      'Total_Increment' : ''}, ignore_index=True)


region_value_all = region_value_all[perf_measure_columns]

In [ ]:
final_df2 = pd.concat([region_value, region_value_all])

In [ ]:
final_df2.to_csv(_join(summary_dir, "A1.10.2_regional_crowding_" + concept_id + '_region' + filename_extension + ".csv"), index=False)

In [ ]:
final_df = pd.concat([final_df1, final_df2], ignore_index=True).reset_index(drop=True)

In [ ]:
final_df.to_csv(_join(summary_dir, "A1.10_regional_crowding_" + concept_id + '_region' + filename_extension + ".csv"), index=False)

#county 
county_df_temp = df_trn_rail.groupby(['orig_county', 'dest_county', 'Period'])['trips'].mean().reset_index()
county_df_temp = county_df_temp.rename(columns={'trips': 'Value',
                                               'orig_county': 'Origin_zone',
                                               'dest_county': 'Dest_zone'})
county_df_temp['Concept_ID'] = concept_id
county_df_temp['Metric_ID'] = 'A1.9'
county_df_temp['Metric_name'] = 'Crowding (Region)'
county_df_temp['Submetric'] = 'A1.9.2'
county_df_temp['Description'] = 'Crowding level between origin and destination county'
county_df_temp['Population'] = 'Whole Population'
county_df_temp['Geography'] = 'County'
county_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
county_df_temp['Units'] = 'minutes'
county_df_temp['Total_Increment'] = ''


#super district
sd_df_temp = df_trn_rail.groupby(['orig_super_dist', 'dest_super_dist', 'Period'])['trips'].mean().reset_index()
sd_df_temp = sd_df_temp.rename(columns={'trips': 'Value',
                                        'orig_super_dist': 'Origin_zone',
                                        'dest_super_dist': 'Dest_zone'})
sd_df_temp['Concept_ID'] = concept_id
sd_df_temp['Metric_ID'] = 'A1.9'
sd_df_temp['Metric_name'] = 'Crowding (Region)'
sd_df_temp['Submetric'] = 'A1.9.3'
sd_df_temp['Description'] = 'Regional crowding level'
sd_df_temp['Population'] = 'Whole Population'
sd_df_temp['Geography'] = 'Super district'
sd_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
sd_df_temp['Units'] = 'minutes'
sd_df_temp['Total_Increment'] = ''


#RDM Zones
rdm_df_temp = df_trn_rail.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Period'])['trips'].mean().reset_index()
rdm_df_temp = rdm_df_temp.rename(columns={'trips': 'Value',
                                        'orig_rdm_zones': 'Origin_zone',
                                        'dest_rdm_zones': 'Dest_zone'})
rdm_df_temp['Concept_ID'] = concept_id
rdm_df_temp['Metric_ID'] = 'A1.9'
rdm_df_temp['Metric_name'] = 'Crowding (Region)'
rdm_df_temp['Submetric'] = 'A1.9.4'
rdm_df_temp['Description'] = 'Regional crowding level'
rdm_df_temp['Population'] = 'Whole Population'
rdm_df_temp['Geography'] = 'RDM'
rdm_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
rdm_df_temp['Units'] = 'minutes'
rdm_df_temp['Total_Increment'] = ''




# transbay region
tb_value = df_trn_rail[df_trn_rail['transbay_od']==1]
tb_value = tb_value.groupby(['Period'])['trips'].mean().reset_index()

tb_value['Concept_ID'] = concept_id
tb_value['Metric_ID'] = 'A1.9'
tb_value['Metric_name'] = 'Crowding (Region)'
tb_value['Submetric'] = 'A1.9.5'
tb_value['Description'] = 'regional crowding level'
tb_value['Population'] = 'Whole Population'
tb_value['Geography'] = 'Transbay'
tb_value['Zone_ID'] = ''
tb_value['Origin_zone'] = ''
tb_value['Dest_zone'] = ''
tb_value['Units'] = 'minutes'
tb_value['Total_Increment'] = ''